# First Hackathon

## 해커톤 진행기간 : 2021.02.09 ~ 2021.02.10


## 진행방향 

1. 데이터 살펴보기
2. 데이터 EDA
3. 모델 구축하기
4. 훈련 및 성능평가
5. 제출

## 후기

으아아아 정찬님 죄송합니다!!    
거의 혼자하신 정찬님께 무한한 감사를

## 데이터 소개

### File descriptions
- sales_train.csv - 훈련데이터, 2013년 1월부터 2015년 10월까지 일별 기록 데이터
- test.csv - 테스트데이터, 2015년 11월 가게, 제품, 매출
- sample_submission.csv - 샘플 제출 파일
- items.csv - 제품 항목에 대한 추가 정보
- item_categories.csv  - 항목 범주에 대한 추가 정보
- shops.csv- 상품에 대한 추가 정보


### Data fields
- ID - 테스트 세트 내의 (Shop, Item) 튜플을 나타내는 ID
- shop_id - shop의 고유 식별자
- item_id - 제품의 고유 식별자
- item_category_id - 항목 범주의 고유 식별자
- item_cnt_day - 판매된 제품 수. 이를 토대로 월별 금액을 예측할 수 있음
- item_price - 품목의 현재 가격
- date - dd/mm/yyyy 형식의 날짜
- date_block_num - 연속 월 번호로, 편의를 위해 사용됩니다. 2013년 1월 = 0, 2013년 2 = 1, ... 2015년 10월 = 33
- item_name - 항목 이름
- shop_name - shop 이름
- item_category_name - 항목 카테고리 이름

- https://www.kaggle.com/dimitreoliveira/deep-learning-for-time-series-forecasting # Deep Learning for Time Series Forecasting
- https://www.kaggle.com/plasticgrammer/future-sales-prediction-playground # Future Sales Prediction: playground

# 참고했던 주소

- https://www.kaggle.com/jagangupta/time-series-basics-exploring-traditional-ts # Time series Basics : Exploring traditional TS
- https://www.kaggle.com/minhtriet/a-beginner-guide-for-sale-data-prediction # A beginner guide for sale data prediction
- https://light-tree.tistory.com/141 # RMSprop이란?
    

In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, date                             # 날짜를 이용할 겁니다
from dateutil.relativedelta import relativedelta                # 시간을 빼고 더해봅시다.

from sklearn.preprocessing import StandardScaler                # 표준화

from math import ceil                                           # 올림함수

from keras.callbacks import LambdaCallback                      # 람다콜백...?
from keras.models import Sequential                             # 시퀀셜??
from keras.layers import Dense, Activation                      # 신경망쌓기, 활성화함수
from keras.layers import LSTM                                   # LSTM모델로 갑시다
from keras.optimizers import RMSprop                            # 기울기 갱신에 씁니다.

%matplotlib inline



In [69]:
# 이야아아아아아ㅏㅇㅇ, 데이터부터 확인합시다.

train = pd.read_csv('/home/aiffel/aiffel/Hackathon/sales_train.csv')
test = pd.read_csv('/home/aiffel/aiffel/Hackathon/test.csv')
submission = pd.read_csv('/home/aiffel/aiffel/Hackathon/sample_submission.csv')
items = pd.read_csv('/home/aiffel/aiffel/Hackathon/items_en.csv')
item_cats = pd.read_csv('/home/aiffel/aiffel/Hackathon/item_categories_en.csv')
shops = pd.read_csv('/home/aiffel/aiffel/Hackathon/shops_en.csv')

In [70]:
#  훈련데이터, 2013년 1월부터 2015년 10월까지 일별 기록 데이터
display(train.tail())  # 연월일 형식 날짜, 날짜 구분 넘버, 가게 넘버, 물건 넘버, 물건 가격, 일일 판매된 제품 수
print(train.shape) # (2935849, 6)
train[train.isnull().any(axis=1)] # 결측치 없다.

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2935844,10.10.2015,33,25,7409,299.0,1.0
2935845,09.10.2015,33,25,7460,299.0,1.0
2935846,14.10.2015,33,25,7459,349.0,1.0
2935847,22.10.2015,33,25,7440,299.0,1.0
2935848,03.10.2015,33,25,7460,299.0,1.0


(2935849, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day


In [71]:
# 테스트데이터, 2015년 11월 가게, 제품, 매출
display(test.head()) # 가게 + 상품 조합 넘버, 가게 넘버, 상품 넘버
print(test.shape)
test[test.isnull().any(axis=1)] # 결측치 없다.

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


(214200, 3)


,ID,shop_id,item_id


In [72]:
# 제출용 샘플 파일 --> Id와 판매량만 넣는다! (2015년 11월만 예측하면 되기 때문)
display(submission.head()) # 가게 + 상품 조합 넘버, 월별 판매된 개수
print(submission.shape)
submission[submission.isnull().any(axis=1)] # 결측치 없다.

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


(214200, 2)


,ID,item_cnt_month


In [73]:
# 제품 항목에 대한 추가 정보
display(items.head()) # 제품명, 제품 넘버, 제품 카테고리 넘버
print(items.shape)
items[items.isnull().any(axis=1)] # 결측치 없다.

,item_name,item_id,item_category_id
0,! POWER IN glamor (PLAST.) D,0,40
1,! ABBYY FineReader 12 Professional Edition Ful...,1,76
2,*** In the glory (UNV) D,2,40
3,*** BLUE WAVE (Univ) D,3,40
4,*** BOX (GLASS) D,4,40


(22170, 3)


,item_name,item_id,item_category_id


In [74]:
# 항목 범주에 대한 추가 정보
display(item_cats.head()) # 제품 항목 이름, 제품 카테고리 넘버
print(item_cats.shape)
item_cats[item_cats.isnull().any(axis=1)] # 결측치 없다.

,item_category_name,item_category_id
0,PC - Headsets / Headphones,0
1,Accessories - PS2,1
2,Accessories - PS3,2
3,Accessories - PS4,3
4,Accessories - PSP,4


(84, 2)


,item_category_name,item_category_id


In [75]:
# 가게에 대한 추가 정보
display(shops.head()) # 가게이름, 가게 넘버
print(shops.shape)
shops[shops.isnull().any(axis=1)] # 결측치 없다.

,shop_name,shop_id
0,"! Yakutsk Ordzhonikidze, 56 Franc",0
1,"! Yakutsk TC ""Central"" Franc",1
2,"Adygea TC ""Mega""",2
3,"Balashikha TRC ""October-Kinomir""",3
4,"Volzhsky mall ""Volga Mall""",4


(60, 2)


,shop_name,shop_id


## 데이터를 보고 정한 EDA 방향

1. date와 date_block_num 중 하나만 써야할 듯하다. ==> 일일 변화량이 의미있는지 파악한 뒤, 아니면 date 삭제
2. shop_id와 item_id 의 조합을 엮어서 하나의 컬럼으로 써야할 듯 하다. ==> ID를 활용하자
3. train 데이터와 test 데이터 사이에 불일치하는 데이터는 삭제해야 할 듯 하다. --> test.unique를 기준으로 train 데이터도 바꾼다.
4. shop_name과 item_name이 필요할까? name컬럼에 대한 고민을 좀 해봐야할 듯 하다.

In [76]:
# 우선 훈련데이터에서 테스트데이터와 공통된 데이터만을 남겨봅시다.
# 테스트데이터에는 ID, shop_id, item_id만이 있습니다. --> train에는 ID가 없으니 제외합니다.
# 이를 기준삼아 train 데이터를 바꿔봅시다.

# 각 컬럼별로 전처리를 진행합니다!

# train데이터 중 test데이터에 있는 shop_id만 남깁니다.
test_shops = test.shop_id.unique()
train = train[train.shop_id.isin(test_shops)]
print("shop_id 전처리 후 :",train.shape) # 2935849 -> 2413246

# train데이터 중 test데이터에 있는 item_id만 남깁니다.
test_items = test.item_id.unique()
train = train[train.item_id.isin(test_items)]
print("item_id 전처리 후 :",train.shape) # 2413246 -> 1224439

train.columns

shop_id 전처리 후 : (2413246, 6)
item_id 전처리 후 : (1224439, 6)


Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price',
       'item_cnt_day'],
      dtype='object')

In [77]:
display(train)
## 흠... shop_id와 item_id를 합쳐서 처음부터 ID 컬럼은 못 만드나? 인덱스로 ID를 주면 더 편할 것 같은데

# key를 만들어서 합쳐볼까?
train['key'] = train['shop_id'].astype(str) + "_" +train['item_id'].astype(str)
test['key'] = test['shop_id'].astype(str) + "_" +test['item_id'].astype(str)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.0,1.0
10,03.01.2013,0,25,2574,399.0,2.0
11,05.01.2013,0,25,2574,399.0,1.0
12,07.01.2013,0,25,2574,399.0,1.0
13,08.01.2013,0,25,2574,399.0,2.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.0,1.0
2935845,09.10.2015,33,25,7460,299.0,1.0
2935846,14.10.2015,33,25,7459,349.0,1.0
2935847,22.10.2015,33,25,7440,299.0,1.0


In [78]:
train = pd.merge(train,test, how='inner', on='key')
train = train.drop(['key','shop_id_y','item_id_y'], axis=1)
train = train.rename(columns={'shop_id_x': 'shop_id', "item_id_x":"item_id"})
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,ID
0,02.01.2013,0,59,22154,999.0,1.0,150983
1,03.01.2013,0,25,2574,399.0,2.0,56520
2,05.01.2013,0,25,2574,399.0,1.0,56520
3,07.01.2013,0,25,2574,399.0,1.0,56520
4,08.01.2013,0,25,2574,399.0,2.0,56520
...,...,...,...,...,...,...,...
1224434,22.10.2015,33,25,7327,349.0,1.0,58974
1224435,24.10.2015,33,25,7315,399.0,1.0,57353
1224436,31.10.2015,33,25,7409,299.0,1.0,58431
1224437,09.10.2015,33,25,7409,299.0,1.0,58431


In [79]:
train[train.isnull().any(axis=1)] # 결측치 없이 잘 붙었다.

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,ID


In [80]:
print("ID의 개수 :",len(train.ID.unique()))
print("shop_id의 개수 :",len(train.shop_id.unique())) # 이걸 기준으로 시각화를 진행한다.
print("item_id의 개수 :",len(train.item_id.unique()))

ID의 개수 : 111404
shop_id의 개수 : 42
item_id의 개수 : 4716


In [86]:
## 그러면 해당 데이터를 가지고 그림을 그려봅시다!!
# 어떤 것을 기준으로 그릴까나==> ID와 date_block_num을 기준으로 그려봅시다 // ID 수가 너무 많아서 fail...
# shop_id로 그려야합니다요..
# 훈련데이터 중에서 shop_id 컬럼과 date_block_num을 기준으로 묶은 뒤, item_cnt_dat를 모두 더합니다. 그 뒤 인덱스를 리셋합니다.
grouped = pd.DataFrame(train.groupby(['shop_id', 'date_block_num'])['item_cnt_day'].sum().reset_index())
display(grouped.head())

# 어떻게 그려야 제일 편안하게 그릴 수 있을까
# x축은 date_block_num으로 한다. --> 이를 통해 시간 흐름에 따른 item_cnt_day를 파악한다.
# 총 33개의 date_block_num이 있다. (train 데이터에서 확인) --> 이것은 하나의 그래프 안에 몽땅 때려넣자

# shop_id는 어떻게 분포가 되어있길래 2~59가 42개냐
shop_list = train.shop_id.unique()
print(sorted(shop_list)) # 특정 몇몇 id가 중간에 빠져있음 이를 고려해서 진행해야 할 듯


,shop_id,date_block_num,item_cnt_day
0,2,0,248.0
1,2,1,135.0
2,2,2,237.0
3,2,3,168.0
4,2,4,159.0


[2, 3, 4, 5, 6, 7, 10, 12, 14, 15, 16, 18, 19, 21, 22, 24, 25, 26, 28, 31, 34, 35, 36, 37, 38, 39, 41, 42, 44, 45, 46, 47, 48, 49, 50, 52, 53, 55, 56, 57, 58, 59]


In [ ]:
# 그래프를 설정해봅시다. 12개의 그래프 (5개씩)를 그리면 됩니다.
fig, axes = plt.subplots(nrows=6, ncols=2, sharex=True, sharey=True, figsize=(16,20))
count = 0


# 햐 이거를 어떻게 5개씩 뽑아서 그리지
for i in range(6):
    for j in range(2):
        sns.pointplot(x='date_block_num', y='item_cnt_day', hue='shop_id',data = , ax=axes[i][j])
        count += 1

In [ ]:
# LSTM으로 예측해봅시다.